In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("alldatahighnosoiltrain.csv" )
data=np.array(data)
X_train=data[0:5804,0:69]
y_train1=data[0:5804,69:129]
y_train2=data[0:5804,129:189]
y_train3=data[0:5804,189:249]
data2 = pd.read_csv("alldatahighnosoiltest.csv" )
data2=np.array(data2)
X_test=data2[0:1451,0:69]
y_test1=data2[0:1451,69:129]
y_test2=data2[0:1451,129:189]
y_test3=data2[0:1451,189:249]
# print(X_train)
print(X_train.shape,y_train1.shape,y_train2.shape,y_train3.shape,X_test.shape,y_test1.shape,y_test2.shape,y_test3.shape)

(5804, 69) (5804, 60) (5804, 60) (5804, 60) (1451, 69) (1451, 60) (1451, 60) (1451, 60)


In [3]:
# DNN
from keras.layers import Input
from keras.models import Model
from keras.utils import plot_model
from kerashypetune import KerasGridSearch
def get_model(param):
    input1 = Input(shape=(69,))
    layer1 = tf.keras.layers.Dense(param['units1'], activation='relu')(input1)
    layer1 = tf.keras.layers.Dense(param['units2'], activation='relu')(layer1)
    outputs1 = tf.keras.layers.Dense(units=60, name='outputs1')(layer1)
    outputs2 = tf.keras.layers.Dense(units=60, name='outputs2')(layer1)
    outputs3 = tf.keras.layers.Dense(units=60, name='outputs3')(layer1)
    model = Model(inputs=[input1], outputs=[outputs1, outputs2, outputs3])
    model.compile(loss='mae', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

In [4]:
# Define the grid of hyperparameters
param_grid = {
'units1': [16, 32, 64, 128],
'units2': [16, 32, 64, 128],
'epochs': 3000,
'batch_size': 32
}
# Create the KerasGridSearch object
kgs = KerasGridSearch(get_model, param_grid, monitor='val_loss', greater_is_better=False)
# Perform the grid search
grid_result = kgs.search(X_train, [y_train1, y_train2, y_train3], validation_data=(X_test, [y_test1, y_test2, y_test3]))


16 trials detected for ('units1', 'units2', 'epochs', 'batch_size')

***** (1/16) *****
Search({'units1': 16, 'units2': 16, 'epochs': 3000, 'batch_size': 32})


KeyboardInterrupt: 

In [ ]:
model_DNN = kgs.best_model
model.save('DNN.h5')

In [ ]:
# CNN
def get_model(param):
    input1 = Input(shape=(69,1))
    layer1 = tf.keras.layers.Conv1D(filters=param['units1'], kernel_size=5, strides=1, padding='same', activation='relu')(input1)
    layer1 = tf.keras.layers.MaxPooling1D(2)(layer1)
    layer1 = tf.keras.layers.Conv1D(filters=param['units2'], kernel_size=5, strides=1, padding='same', activation='relu')(layer1)
    layer1 = tf.keras.layers.MaxPooling1D(2)(layer1)
    layer1 = tf.keras.layers.Flatten()(layer1)
    layer1 = tf.keras.layers.Dense(128,activation='relu')(layer1)
    layer1 = tf.keras.layers.Dense(128,activation='relu')(layer1)
    outputs1 = tf.keras.layers.Dense(units=60, name='outputs1')(layer1)
    outputs2 = tf.keras.layers.Dense(units=60, name='outputs2')(layer1)
    outputs3 = tf.keras.layers.Dense(units=60, name='outputs3')(layer1)
    model = Model(inputs=[input1],outputs=[outputs1,outputs2,outputs3])
    model.compile(loss='mae', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

In [ ]:
# Define the grid of hyperparameters
param_grid = {
'units1': [16, 32, 64, 128],
'units2': [16, 32, 64, 128],
'epochs': 3000,
'batch_size': 32
}
# Create the KerasGridSearch object
kgs = KerasGridSearch(get_model, param_grid, monitor='val_loss', greater_is_better=False)
# Perform the grid search
grid_result = kgs.search(X_train, [y_train1, y_train2, y_train3], validation_data=(X_test, [y_test1, y_test2, y_test3]))

In [ ]:
model_CNN = kgs.best_model
model.save('CNN.h5')

In [ ]:
# LSTM
def get_model(param):
    input1 = Input(shape=(69,1))
    layer1 = tf.keras.layers.LSTM(param['units1'], activation='relu', return_sequences=True)(input1)
    layer1 = tf.keras.layers.Flatten()(layer1)
    layer1 = tf.keras.layers.Dense(128,activation='relu')(layer1)
    layer1 = tf.keras.layers.Dense(128,activation='relu')(layer1)
    outputs1 = tf.keras.layers.Dense(units=60, name='outputs1')(layer1)
    outputs2 = tf.keras.layers.Dense(units=60, name='outputs2')(layer1)
    outputs3 = tf.keras.layers.Dense(units=60, name='outputs3')(layer1)
    model = Model(inputs=[input1],outputs=[outputs1,outputs2,outputs3])
    print(model.summary())
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mae',metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

In [ ]:
# Define the grid of hyperparameters
param_grid = {
'units1': [16, 32, 64, 128],
'epochs': 3000,
'batch_size': 32
}
# Create the KerasGridSearch object
kgs = KerasGridSearch(get_model, param_grid, monitor='val_loss', greater_is_better=False)
# Perform the grid search
grid_result = kgs.search(X_train, [y_train1, y_train2, y_train3], validation_data=(X_test, [y_test1, y_test2, y_test3]))

In [ ]:
model_LSTM = kgs.best_model
model.save('LSTM.h5')